In [111]:
import os, sys
import json
import cv2
import pickle
import pydaisi as pyd
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from dotenv import load_dotenv
load_dotenv()

automatic_cheque_clearing_system = pyd.Daisi("soul0101/Automatic Cheque Clearing System")

[17:39:58] INFO     Calling https://app.daisi.io/pebble-api/daisies/connect?name=soul0101/Automatic  ]8;id=572595;file://c:\Users\91983\anaconda3\envs\face\lib\site-packages\pydaisi\pydaisi.py\pydaisi.py]8;;\:]8;id=282118;file://c:\Users\91983\anaconda3\envs\face\lib\site-packages\pydaisi\pydaisi.py#411\411]8;;\
                    Cheque Clearing System                                                                         

[17:40:00] INFO     Found existing Daisi: f8d428cf-2c93-410e-9aa9-456e99b48516                       ]8;id=473149;file://c:\Users\91983\anaconda3\envs\face\lib\site-packages\pydaisi\pydaisi.py\pydaisi.py]8;;\:]8;id=195594;file://c:\Users\91983\anaconda3\envs\face\lib\site-packages\pydaisi\pydaisi.py#424\424]8;;\

In [112]:
raw_image = np.array(Image.open('./images/icici1_tilted.png'))
segmentation_template = json.loads(open('./templates/icici.json', "r").read())
raw_extraction_result_pipeline, sanitized_result_pipeline, segments_pipeline = automatic_cheque_clearing_system.cheque_verification_pipeline(raw_image, segmentation_template=segmentation_template).value
print(sanitized_result_pipeline)

{'micr_strip': '⑈100830⑈500229009⑆551452⑈31', 'acc_number': '630801551452', 'payee': 'B. Shiva Kumar', 'courtesy_amount': 21400000, 'legal_amount': 'Two crore four lakh', 'date_box': '08-02-2016'}


In [113]:
with open('./results/raw_extraction_result_pipeline.pickle', 'rb') as handle:
    raw_extraction_result_pipeline_gold = pickle.load(handle)
if raw_extraction_result_pipeline != raw_extraction_result_pipeline_gold:
    sys.exit(1)

with open('./results/sanitized_result_pipeline.pickle', 'rb') as handle:
    sanitized_result_pipeline_gold = pickle.load(handle)
if sanitized_result_pipeline != sanitized_result_pipeline_gold:
    sys.exit(1)

with open('./results/segments_pipeline.pickle', 'rb') as handle:
    segments_pipeline_gold = pickle.load(handle)
for key, value in segments_pipeline_gold.items():
    if(np.any(segments_pipeline[key] != segments_pipeline_gold[key])):
        sys.exit(1)

In [114]:
cropped_img = automatic_cheque_clearing_system.crop_to_cheque(raw_image).value

In [115]:
with open('./results/cropped_img.pickle', 'rb') as handle:
    cropped_img_gold = pickle.load(handle)

if np.any(cropped_img != cropped_img_gold):
    sys.exit(1)

In [116]:
cleaned_img = automatic_cheque_clearing_system.clean_cheque(cropped_img).value

In [117]:
with open('./results/cleaned_img.pickle', 'rb') as handle:
    cleaned_img_gold = pickle.load(handle)

if np.any(cleaned_img != cleaned_img_gold):
    sys.exit(1)

In [118]:
segmentation_result = automatic_cheque_clearing_system.segment_cheque(cleaned_img, segmentation_template=segmentation_template).value

In [119]:
with open('./results/segmentation_result.pickle', 'rb') as handle:
    segmentation_result_gold = pickle.load(handle)

for key, value in segmentation_result_gold.items():
    if(np.any(segmentation_result[key] != segmentation_result_gold[key])):
        sys.exit(1)

In [120]:
raw_extraction_result = automatic_cheque_clearing_system.azure_cheque_ocr(cleaned_img, segmentation_template=segmentation_template).value

In [121]:
with open('./results/raw_extraction_result.pickle', 'rb') as handle:
    raw_extraction_result_gold = pickle.load(handle)

if raw_extraction_result != raw_extraction_result_gold:
    sys.exit(1)

In [122]:
sanitized_result = automatic_cheque_clearing_system.extraction_result_sanitizer(raw_extraction_result).value

In [123]:
with open('./results/sanitized_result.pickle', 'rb') as handle:
    sanitized_result_gold = pickle.load(handle)

if sanitized_result != sanitized_result_gold:
    sys.exit(1)

In [124]:
orig_sign_np = cv2.imread('../images/original_sign.png')
check_sign_np = cv2.imread('../images/original_sign.png')
cleaned_orig_sign, cleaned_check_sign = automatic_cheque_clearing_system.signature_cleaner([orig_sign_np, check_sign_np]).value

In [125]:
with open('./results/cleaned_signatures.pickle', 'rb') as handle:
    cleaned_orig_sign_gold, cleaned_check_sign_gold = pickle.load(handle)

if np.any(cleaned_orig_sign != cleaned_orig_sign_gold) or np.any(cleaned_check_sign != cleaned_check_sign_gold):
    sys.exit(1)

In [126]:
verification_result = automatic_cheque_clearing_system.verify_signatures(orig_sign_np, check_sign_np).value

In [127]:
with open('./results/verification_result.pickle', 'rb') as handle:
    verification_result_gold = pickle.load(handle)

if verification_result != verification_result_gold:
    sys.exit(1)